# Лабораторная работа 10. Матчасть DL

Задача: реализовать и обучить нейронную сеть, состоящую из 2 нейронов, предсказывать значения функции XOR. При выполнении лабораторной запрещается использовать фреймворки для глубокого обучения (как PyTorch, Tensorflow, Caffe, Theano и им подобные).

В первую очередь ознакомиться с [этим](https://towardsdatascience.com/implementing-the-xor-gate-using-backpropagation-in-neural-networks-c1f255b4f20d) материалом.

Что необходимо реализовать, используя знания и фрагменты кода из ссылки выше:
1. Класс Neuron, имеющий вектор весов self._weigths
2. Два метода класса Neuron: forward(x), backward(x, loss) - реализующих прямой и обратный проход по нейронной сети.
   Метод forward должен реализовывать логику работу нейрона: умножение входа на вес self._weigths, сложение и функцию активации сигмоиду.
   Метод backward должен реализовывать взятие производной от сигмоиды и используя состояние нейрона обновить его веса.
3. Реализовать с помощью класса Neuron нейронную сеть с архитектурой из трёх нейронов, предложенную в статье.

    Для красоты обернуть в класс Model с методами forward и backward, реализующими правильное взаимодействие нейронов на прямом и обратном проходах.
4. Реализовать тренировочный цикл следующего вида:

```
цикл (обучающие данные):
	y = model.forward(x)
	err = loss(y, label)
	model.backward(x, err)
```
В итоге обучения должны предсказываться значения аналогичные описанным в опорной статье.

*Та же лаба человеческим языком:**
Взять код из статьи, обернуть его в ООП. Попутно разобраться как работает прямое и обратное распространение.

[Доп.материал](https://habr.com/ru/post/313216/)

In [17]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    return x * (1 - x)


def loss(predicted, expected):
    return expected - predicted

In [18]:
class Neuron:
    def __init__(self, input_num, output_num, activ_func=sigmoid, loss_func=sigmoid_derivative, lr=0.1):
        #Random weights and biases initialization
        self.W = np.random.uniform(-1, 1, (input_num, output_num))  # Weights
        self.b = np.random.uniform(-1, 1, (1, output_num))  # Biases
        self.activ_func = activ_func
        self.loss_func = loss_func
        self.lr = lr

    def forward(self, X):
        self.otput_value = self.activ_func(X @ self.W + self.b)  # sigma(XW+b)
        return self.otput_value

    def delta(self, error):
        return error * self.loss_func(self.otput_value)  # delta = error * sigma'(XW+b)

    def backward(self, x, delta):  # update weights and biases
        self.W += x.T @ delta * self.lr
        self.b += np.sum(delta, axis=0) * self.lr

In [19]:
class Model:
    def __init__(self, input_num, hidden_num, output_num, activ_func=sigmoid, loss_func=sigmoid_derivative, lr=0.1):
        self.layer_h = Neuron(input_num, hidden_num, activ_func, loss_func, lr)  # Hidden Layer (2, 2)
        self.layer_o = Neuron(hidden_num, output_num, activ_func, loss_func, lr)  # Output Layer (2, 1)

    def forward(self, x):
        self.output_h = self.layer_h.forward(x)
        return self.layer_o.forward(self.output_h)

    def backward(self, x, err):
        delta_o = self.layer_o.delta(err)  # output neuron delta
        self.layer_o.backward(self.output_h, delta_o)  # update weights of output neuron

        error_h = delta_o.dot(self.layer_o.W.T)  # error at the output of hidden neurons
        delta_h = self.layer_h.delta(error_h)  # hidden neurons delta
        self.layer_h.backward(x, delta_h)  # update weights of hidden neurons

In [20]:
#Input datasets
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
expected_output = np.array([[0], [1], [1], [0]])

In [23]:
epochs = 10000
np.random.seed(19193)

model = Model(2, 2, 1)

print("Initial hidden weights: ",end='')
print(*model.layer_h.W)
print("Initial hidden biases: ",end='')
print(*model.layer_h.b)
print("Initial output weights: ",end='')
print(*model.layer_o.W)
print("Initial output biases: ",end='')
print(*model.layer_o.b)

#Training algorithm
for _ in (range(epochs)):
	predicted_output = model.forward(inputs)
	err = loss(predicted_output, expected_output)
	model.backward(inputs, err)

print("Final hidden weights: ",end='')
print(*model.layer_h.W)
print("Final hidden bias: ",end='')
print(*model.layer_h.b)
print("Final output weights: ",end='')
print(*model.layer_o.W)
print("Final output bias: ",end='')
print(*model.layer_o.b)

print("\nOutput from neural network after 10,000 epochs: ",end='')
print(*predicted_output)

Initial hidden weights: [-0.51801296 -0.43439121] [ 0.48013776 -0.4310847 ]
Initial hidden biases: [-0.93001131  0.7195042 ]
Initial output weights: [0.30274019] [0.90186948]
Initial output biases: [0.42651689]
Final hidden weights: [-5.56045937  4.55158174] [ 5.66032357 -4.88516238]
Final hidden bias: [-3.32126841 -2.52868743]
Final output weights: [7.06812462] [7.20454438]
Final output bias: [-3.5162015]

Output from neural network after 10,000 epochs: [0.0607929] [0.94953011] [0.94519298] [0.05439818]


Оригинальный код из статьи

In [22]:
import numpy as np
np.random.seed(0)

def sigmoid (x):
    return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

#Input datasets
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
expected_output = np.array([[0],[1],[1],[0]])

epochs = 10000
lr = 0.1
inputLayerNeurons, hiddenLayerNeurons, outputLayerNeurons = 2,2,1

#Random weights and bias initialization
hidden_weights = np.random.uniform(size=(inputLayerNeurons,hiddenLayerNeurons))
hidden_bias =np.random.uniform(size=(1,hiddenLayerNeurons))
output_weights = np.random.uniform(size=(hiddenLayerNeurons,outputLayerNeurons))
output_bias = np.random.uniform(size=(1,outputLayerNeurons))

print("Initial hidden weights: ",end='')
print(*hidden_weights)
print("Initial hidden biases: ",end='')
print(*hidden_bias)
print("Initial output weights: ",end='')
print(*output_weights)
print("Initial output biases: ",end='')
print(*output_bias)


#Training algorithm
for _ in range(epochs):
	#Forward Propagation
	hidden_layer_activation = np.dot(inputs,hidden_weights)
	hidden_layer_activation += hidden_bias
	hidden_layer_output = sigmoid(hidden_layer_activation)

	output_layer_activation = np.dot(hidden_layer_output,output_weights)
	output_layer_activation += output_bias
	predicted_output = sigmoid(output_layer_activation)

	#Backpropagation
	error = expected_output - predicted_output
	d_predicted_output = error * sigmoid_derivative(predicted_output)

	error_hidden_layer = d_predicted_output.dot(output_weights.T)
	d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

	#Updating Weights and Biases
	output_weights += hidden_layer_output.T.dot(d_predicted_output) * lr
	output_bias += np.sum(d_predicted_output,axis=0,keepdims=True) * lr
	hidden_weights += inputs.T.dot(d_hidden_layer) * lr
	hidden_bias += np.sum(d_hidden_layer,axis=0,keepdims=True) * lr

print("Final hidden weights: ",end='')
print(*hidden_weights)
print("Final hidden bias: ",end='')
print(*hidden_bias)
print("Final output weights: ",end='')
print(*output_weights)
print("Final output bias: ",end='')
print(*output_bias)

print("\nOutput from neural network after 10,000 epochs: ",end='')
print(*predicted_output)

Initial hidden weights: [0.5488135  0.71518937] [0.60276338 0.54488318]
Initial hidden biases: [0.4236548  0.64589411]
Initial output weights: [0.43758721] [0.891773]
Initial output biases: [0.96366276]
Final hidden weights: [3.70986741 5.78826314] [3.71149043 5.79621929]
Final hidden bias: [-5.6837777  -2.42016264]
Final output weights: [-8.14923676] [7.50521163]
Final output bias: [-3.37826206]

Output from neural network after 10,000 epochs: [0.05770383] [0.9470198] [0.9469948] [0.05712647]
